In [1]:
import timm

# Load mô hình ViT
model = timm.create_model('vit_base_patch16_224', pretrained=True)
print(model)

/home/hieuld/Documents/vit_npu/python/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [2]:
import torch
import timm
import numpy as np

# Load mô hình Vision Transformer
model = timm.create_model('vit_base_patch16_224', pretrained=True)
model.eval()

# Tạo input giả lập (batch=1, 3 kênh màu, 224x224)
dummy_input = torch.randn(1, 3, 224, 224)

# Danh sách lưu giá trị Q, K, V
QKV_list = []

# Hook function để lấy giá trị từ qkv
def hook_fn(module, input, output):
    QKV_list.append(output)

# Tìm module `qkv` bên trong Attention
for name, module in model.named_modules():
    if name.endswith("attn.qkv"):
        module.register_forward_hook(hook_fn)

# Chạy mô hình để lấy Q, K, V
with torch.no_grad():
    model(dummy_input)

# Tách tensor QKV
QKV_tensor = QKV_list[0]  # (batch, num_patches, head_dim * 3)
batch_size, num_patches, head_dim_3 = QKV_tensor.shape
head_dim = head_dim_3 // 3

Q = QKV_tensor[:, :, :head_dim]
K = QKV_tensor[:, :, head_dim:2*head_dim]
V = QKV_tensor[:, :, 2*head_dim:]

print(f"Q shape: {Q.shape}, K shape: {K.shape}, V shape: {V.shape}")


Q shape: torch.Size([1, 197, 768]), K shape: torch.Size([1, 197, 768]), V shape: torch.Size([1, 197, 768])


In [ ]:
# Chuyển đổi sang fixed-point Q1.15
scale_factor = 2**15  # Q1.15: 1 đơn vị float = 2^15 fixed-point
Q_fixed = np.round(Q.numpy() * scale_factor).astype(np.int16)
K_fixed = np.round(K.numpy() * scale_factor).astype(np.int16)
V_fixed = np.round(V.numpy() * scale_factor).astype(np.int16)

# Xuất ra file .mem để dùng trên FPGA
np.savetxt("Q_fixed.mem", Q_fixed.reshape(-1), fmt="%d")
np.savetxt("K_fixed.mem", K_fixed.reshape(-1), fmt="%d")
np.savetxt("V_fixed.mem", V_fixed.reshape(-1), fmt="%d")

print("Dữ liệu Q, K, V đã được lưu thành công!")

Dữ liệu Q, K, V đã được lưu thành công!


In [6]:
Q_fixed.shape
K_fixed.shape
V_fixed.shape

(1, 197, 768)